<a href="https://colab.research.google.com/github/ImranRiazChohan/Cat-and-Dog-Classification-CNN-/blob/master/Cat_and_Dog_Classification(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

In [0]:
import zipfile
local_zip = '/content/cat-and-dog.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content')
zip_ref.close()

In [0]:
base_dir1="/content/cat-and-dog"